In [2]:
import cv2 as cv
import mediapipe as mp

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils
# fungsi untuk mendeteksi tangan dengan media pipe
def detect_hand_gesture(image,hand):
    imageRGB = cv.cvtColor(image,cv.COLOR_BGR2RGB)
    res  = hand.process(imageRGB)
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            gesture = recognize_gesture(hand_landmarks)
            mp_draw.draw_landmarks(image,hand_landmarks,mp_hands.HAND_CONNECTIONS)
            cv.putText(image,gesture,(50,50),cv.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv.LINE_AA)
    return image

def recognize_gesture(hand_landmarks):
    # ambil posisi ujung jari
    ujung_jempol = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    ujung_telunjuk = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    ujung_tengah = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ujung_manis = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    ujung_kelingking = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # jempol kang
    if(
        ujung_jempol.y < ujung_telunjuk.y and
        ujung_jempol.y < ujung_tengah.y and
        ujung_jempol.y < ujung_manis.y and
        ujung_jempol.y < ujung_kelingking.y
    ): return "jempol"
    if(
        ujung_tengah.y < ujung_manis.y and
        ujung_tengah.y < ujung_jempol.y and
        ujung_tengah.y < ujung_kelingking.y and
        ujung_tengah.y < ujung_telunjuk.y 
    ) : return "fucek samid"
    return "gesture tidak diketahui"


frameWidth = 320
frameHeight = 720
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_HEIGHT,frameHeight)
cap.set(cv.CAP_PROP_FRAME_WIDTH,frameWidth)
if not cap.isOpened():
    print("tidak dapat membuka kamera")
while(cap.isOpened):
    ret,frame = cap.read()
    if not ret:
        print("gagal membuka kamera")
        break;
    frame = detect_hand_gesture(frame,hands)
    cv.imshow("hand gasture", frame)
    if cv.waitKey(1) & 0xff ==ord('q'):
        break
cap.release()
cv.destroyAllWindows()